In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load all the required libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score as f1
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score
import scikitplot as skplt
from collections import Counter

plt.rc('figure',figsize=(18,9))
%pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

## Load the dataset

In [ ]:
credit_data = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
credit_data = credit_data[credit_data.columns[:-2]]
credit_data.head(3)

In [ ]:
X = credit_data.drop('Attrition_Flag', axis=1)
y = credit_data['Attrition_Flag']

## Exploratory Data Analysis

1. **Age distribution**

In [ ]:
plt.figure(figsize = (18,10))
plt.title("Age with Churned or not", fontsize = 30)
sns.countplot(data = credit_data, x = credit_data["Customer_Age"], hue = "Attrition_Flag")

We can see that the distribution of customer ages in our dataset follows a fairly normal distribution, thus further use of the age feature can be done with the normality assumption.

**2. Percentage of existing and attrited customers**

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
count = Counter(y)
ax.pie(count.values(), labels=count.keys(), autopct=lambda p:f'{p:.2f}%')
ax.set_title('Percentage of existing and attrited/churned customers')
plt.show()

As we can see only 16% of the data samples represent churn customers, so we will use SMOTE to upsample the churn samples to match them with the regular customer sample size in order to give the later selected models a better chance of catching on small details which will almost definitely be missed out with such a size difference.

**3. Gender wise Existing and Churned customers**

In [ ]:
#countplot
plt.figure(figsize = (12,8))
plt.title("Gender wise Existing and Churned customers")
sns.countplot(data = credit_data, x = credit_data["Gender"], hue = "Attrition_Flag")

**4. Education Level in Existing and Churned Customers**

In [ ]:
gender = credit_data.loc[credit_data["Attrition_Flag"] == "Attrited Customer", ["Education_Level"]].value_counts()
gender_normal = credit_data.loc[credit_data["Attrition_Flag"] == "Attrited Customer", ["Education_Level"]].value_counts().tolist()
gender_churned = credit_data.loc[credit_data["Attrition_Flag"] == "Existing Customer", ["Education_Level"]].value_counts().tolist()
fig, ax = plt.subplots(1, 2, dpi = 200, figsize = (15,8))
ax[0].set_title("Education Level in Existing Customers")
ax[0].pie(x = gender_normal, labels = gender.index, autopct='%.2f%%')
ax[1].set_title("Education Level in Churned Customers")
ax[1].pie(x = gender_churned, labels = gender.index, autopct='%.2f%%')

The features in our dataset have some correlation, for example, Months_on_book and Customer_age, Avg_Utilization_Ratio and Total_Revolving_Bal.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(X.corr(), annot=True)
plt.show()

## DATA PROCESSING


Convert all the categorical variables into numerical variable

In [ ]:
# #--------------let's convert some categorical variables into numerical--------------

# #ordinal to numerical
map_education_level = {'High School':1,'Graduate':3,'Uneducated':0,'College':2,'Post-Graduate':4,'Doctorate':5}
map_income_level = {'$60K - $80K':3,'Less than $40K':1, '$80K - $120K':4,'$40K - $60K':2,'$120K +':5}
map_card_category = {'Blue':1,'Gold':3,'Silver':2,'Platinum':4}
X['Education_Level'].replace(map_education_level,inplace=True)
X['Income_Category'].replace(map_income_level,inplace=True)
X['Card_Category'].replace(map_card_category,inplace=True)


# #hot encoding of gender category
X.insert(2,'Gender_M',X['Gender'],True)
X.rename({'Gender':'Gender_F'},axis=1,inplace=True)
X['Gender_M'].replace({'M':1,'F':0},inplace=True)
X['Gender_F'].replace({'M':0,'F':1},inplace=True)


# #hot encoding of marital status
X.insert(7,'Single',X['Marital_Status'],True)
X.insert(7,'Divorced',X['Marital_Status'],True)
X.insert(7,'Unknown',X['Marital_Status'],True)
X.rename({'Marital_Status':'Married'},axis=1,inplace=True)
X['Married'].replace({'Single':0, 'Married':1, 'Divorced':0, 'Unknown':0},inplace=True)
X['Single'].replace({'Single':1, 'Married':0, 'Divorced':0, 'Unknown':0},inplace=True)
X['Divorced'].replace({'Single':0, 'Married':0, 'Divorced':1, 'Unknown':0},inplace=True)
X['Unknown'].replace({'Single':0, 'Married':0, 'Divorced':0, 'Unknown':1},inplace=True)


y.replace({'Existing Customer':0, 'Attrited Customer':1},inplace=True)

X.head()

## Handling missing values

In [ ]:
plt.hist(X.loc[X['Income_Category']!='Unknown']['Income_Category'])   # income is rightly skewed. so central value is median
plt.show()

plt.hist(X.loc[X['Education_Level']!='Unknown']['Education_Level'])   # education is normally distributed. so central value is mean
plt.show()

#Missing values in education column
educatedDF = X.loc[X['Education_Level']!='Unknown']
mean_education = educatedDF['Education_Level'].mean()
X['Education_Level'].replace({'Unknown':mean_education},inplace=True)

#Missing values in income column
salariedDF = X.loc[X['Income_Category']!='Unknown']
median_salaries = salariedDF['Income_Category'].median()
X['Income_Category'].replace({'Unknown':median_salaries},inplace=True)

X.head()

## Data Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

## Scaling Numeric Values

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Upsampling using SMOTE

In [ ]:
#-----Upsampling----
from sklearn.utils import resample
from collections import Counter

print("Before Upsampling:-")
print(Counter(y_train))


# Let's use SMOTE to oversample
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train_upsampled, y_train_upsampled = oversample.fit_resample(X_train,y_train)

print("After Upsampling:-")
print(Counter(y_train_upsampled))

## Building Classification models


1. Random Forest

In [ ]:
classifier = RandomForestClassifier(n_estimators = 50, random_state = 0)
classifier.fit(X_train_upsampled, y_train_upsampled)

# Predicting result for training set and validation set
predict_val_rf = classifier.predict(X_test)

# Model Performance
print("Accuracy : ", accuracy_score(y_test, predict_val_rf) *  100)
print("Recall : ", recall_score(y_test, predict_val_rf) *  100)
print("Precision : ", precision_score(y_test, predict_val_rf) *  100)
print(confusion_matrix(y_test, predict_val_rf))
print(classification_report(y_test, predict_val_rf))

2. XG Boost

In [ ]:
model = XGBClassifier()
model.fit(X_train_upsampled, y_train_upsampled)

# Predicting result for training set and validation set
predict_val_rf2 = model.predict(X_test)

# Model Performance
print("Accuracy : ", accuracy_score(y_test, predict_val_rf2) *  100)
print("Recall : ", recall_score(y_test, predict_val_rf2) *  100)
print("Precision : ", precision_score(y_test, predict_val_rf2) *  100)
print(confusion_matrix(y_test, predict_val_rf2))
print(classification_report(y_test, predict_val_rf2))

3. Naive Bayes

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train_upsampled, y_train_upsampled)

# Predicting result for training set and validation set
predict_val_rf2 = gaussian.predict(X_test)

# Model Performance
print("Accuracy : ", accuracy_score(y_test, predict_val_rf2) *  100)
print("Recall : ", recall_score(y_test, predict_val_rf2) *  100)
print("Precision : ", precision_score(y_test, predict_val_rf2) *  100)
print(confusion_matrix(y_test, predict_val_rf2))
print(classification_report(y_test, predict_val_rf2))

4. Support Vector Machines

In [ ]:
svc = SVC()
svc.fit(X_train_upsampled, y_train_upsampled)

# Predicting result for training set and validation set
predict_val_rf2 = svc.predict(X_test)

# Model Performance
print("Accuracy : ", accuracy_score(y_test, predict_val_rf2) *  100)
print("Recall : ", recall_score(y_test, predict_val_rf2) *  100)
print("Precision : ", precision_score(y_test, predict_val_rf2) *  100)
print(confusion_matrix(y_test, predict_val_rf2))
print(classification_report(y_test, predict_val_rf2))